<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("OECD_EO.dta")
stata_df.head()

,period,period_start_day,value,frequency,dataset_code,dataset_name,indicator,location,indexed_at,provider_code,series_code,series_name,series_num
0,1970,1970-01-01,-1.528287,annual,NAAG,National Accounts at a Glance,B9S13S,AUS,2024-07-03T07:12:51.547Z,OECD,AUS.B9S13S,"Australia – Net lending/net borrowing, General...",Master
1,1971,1971-01-01,-1.495050,annual,NAAG,National Accounts at a Glance,B9S13S,AUS,2024-07-03T07:12:51.547Z,OECD,AUS.B9S13S,"Australia – Net lending/net borrowing, General...",Master
2,1972,1972-01-01,-1.961060,annual,NAAG,National Accounts at a Glance,B9S13S,AUS,2024-07-03T07:12:51.547Z,OECD,AUS.B9S13S,"Australia – Net lending/net borrowing, General...",Master
3,1973,1973-01-01,-0.826679,annual,NAAG,National Accounts at a Glance,B9S13S,AUS,2024-07-03T07:12:51.547Z,OECD,AUS.B9S13S,"Australia – Net lending/net borrowing, General...",Master
4,1974,1974-01-01,-3.445372,annual,NAAG,National Accounts at a Glance,B9S13S,AUS,2024-07-03T07:12:51.547Z,OECD,AUS.B9S13S,"Australia – Net lending/net borrowing, General...",Master


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [2]:
python_df = pd.read_pickle("OECD_EO.pkl")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,LOCATION,INDICATOR,Country,Indicator
0,annual,OECD,EO,Economic Outlook No 115 - May 2024,ARG.CBGDPR.A,Argentina – Current account balance as a perce...,1994,1994-01-01,-3.931945,-3.931945,ARG,CBGDPR,Argentina,Current account balance as a percentage of GDP
1,annual,OECD,EO,Economic Outlook No 115 - May 2024,ARG.CBGDPR.A,Argentina – Current account balance as a perce...,1995,1995-01-01,-1.826127,-1.826127,ARG,CBGDPR,Argentina,Current account balance as a percentage of GDP
2,annual,OECD,EO,Economic Outlook No 115 - May 2024,ARG.CBGDPR.A,Argentina – Current account balance as a perce...,1996,1996-01-01,-2.290679,-2.290679,ARG,CBGDPR,Argentina,Current account balance as a percentage of GDP
3,annual,OECD,EO,Economic Outlook No 115 - May 2024,ARG.CBGDPR.A,Argentina – Current account balance as a perce...,1997,1997-01-01,-3.817375,-3.817375,ARG,CBGDPR,Argentina,Current account balance as a percentage of GDP
4,annual,OECD,EO,Economic Outlook No 115 - May 2024,ARG.CBGDPR.A,Argentina – Current account balance as a perce...,1998,1998-01-01,-4.464039,-4.464039,ARG,CBGDPR,Argentina,Current account balance as a percentage of GDP


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [3]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [4]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce').astype('float64')
python_df['value'] = python_df['value'].astype('float64')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [5]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [6]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True